In [18]:
import pyodbc

In [152]:
import requests as r
import pandas as pd
import json
from sqlalchemy import create_engine
import ast
import time

In [186]:
login_url='https://api.nb.com.ar/v1/auth/login'
nb_user='RSHA38'
nb_pass='b1wjgp6nTm3'

login_body = {'user':nb_user,'password':nb_pass,'mode':'api'}

response=r.post(login_url,json=login_body)

token=response.json()['token']

In [187]:
#products_url='https://api.nb.com.ar/v1/'
categories_url='https://api.nb.com.ar/v1/categories'
categories_auth= {'Authorization':'Bearer '+token}
categories_response=r.get(categories_url,headers=categories_auth)

In [188]:
categories = pd.read_json(categories_response.text,orient='records')


C:\Users\Lautaro\AppData\Local\Temp\ipykernel_23792\1667891171.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  categories = pd.read_json(categories_response.text,orient='records')


In [210]:
products_url = f'https://api.nb.com.ar/v1/category/TECLADOS?order_a_z=1'
products_auth= {'Authorization':'Bearer '+token}
products_response=r.get(products_url,headers=products_auth)
products_response.text

'[{"title":"TECLADO GAMER RAZER CYNOSA LITE SPANISH","sku":"RZ03-02741600-R311","id":104406,"category":"TECLADOS","categoryId":34,"brand":"RAZER","brandId":85,"mainImage":"https:\\/\\/static.nb.com.ar\\/i\\/nb_TECLADO-GAMER-RAZER-CYNOSA-LITE-SPANISH_ver_f83e62586d6ee9d8459ed8a07417fc9f.jpeg","brandImage":"https:\\/\\/static.nb.com.ar\\/img\\/d1c3fc2aacd139c11b386f8ee766b995.jpg","initialB":5,"initialC":10,"stock":"Sin stock","amountInCart":0,"amountStock":0,"categoryIdUser":null,"categoryDescriptionUser":null,"utility":null,"highAverage":150,"widthAverage":400,"lengthAverage":30,"weightAverage":600,"price":{"value":43.94144,"iva":10.5,"finalPrice":48.5552912,"percepcion":null,"finalPriceWithUtility":48.55529,"ncostoextra":0},"warranty":"12 meses","cotizacion":950.5},{"title":"ACCESORIO TECLADO TT ESPORTS TECLAS METALCAPS 38 PIEZAS","sku":"","id":7430,"category":"TECLADOS","categoryId":34,"brand":"TT ESPORTS","brandId":72,"mainImage":"https:\\/\\/static.nb.com.ar\\/i\\/nb_ACCESORIO-TECL

In [225]:
product_response_list=[]
df=pd.DataFrame()
categories_failed=[]
for category in ast.literal_eval(categories_response.text):
    #products_url=f'https://api.nb.com.ar/v1/?category={category['description']}'
    products_url = f'https://api.nb.com.ar/v1/category/{str(category['description']).strip()}?order_a_z=1'
    products_auth= {'Authorization':'Bearer '+token}
    products_response=r.get(products_url,headers=products_auth)

    if str(products_response)=='<Response [200]>':
        df_it=pd.read_json(products_response.text)
        df=pd.concat([df,df_it],ignore_index=True)
        #time.sleep(2)
    else:
        categories_failed.append(str(category['description']))

C:\Users\Lautaro\AppData\Local\Temp\ipykernel_23792\3219917200.py:11: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_it=pd.read_json(products_response.text)
C:\Users\Lautaro\AppData\Local\Temp\ipykernel_23792\3219917200.py:11: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_it=pd.read_json(products_response.text)
C:\Users\Lautaro\AppData\Local\Temp\ipykernel_23792\3219917200.py:11: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_it=pd.read_json(products_response.text)
C:\Users\Lautaro\AppData\Local\Temp\ipykernel_23792\3219917200.py:11: FutureWarning: Passing literal json to 'read_json' is deprecated and will be r

In [239]:
products_response.text


'[{"title":"CAMARA TRUST IRIS CONFERENCIA","sku":"24073","id":117309,"category":"WEBCAMS","categoryId":24,"brand":"TRUST","brandId":128,"mainImage":"https:\\/\\/static.nb.com.ar\\/i\\/nb_CAMARA-TRUST-IRIS-CONFERENCIA_ver_74fa73ccbcb03cfc905e7e68ee86240d.png","brandImage":"https:\\/\\/static.nb.com.ar\\/img\\/0d580a29c679330cff1810970adca7da.png","initialB":5,"initialC":10,"stock":"Bajo","amountInCart":0,"amountStock":1,"categoryIdUser":null,"categoryDescriptionUser":null,"utility":null,"highAverage":200,"widthAverage":100,"lengthAverage":200,"weightAverage":300,"price":{"value":1103.45767,"iva":21,"finalPrice":1335.1837807,"percepcion":null,"finalPriceWithUtility":1335.18378,"ncostoextra":0},"warranty":"12 meses","cotizacion":950.5},{"title":"OUTLET WEBCAM GENIUS S RS ECAM 8000 RED NEW","sku":"O32200001407","id":110950,"category":"WEBCAMS","categoryId":24,"brand":"GENIUS","brandId":47,"mainImage":"https:\\/\\/static.nb.com.ar\\/i\\/nb_OUTLET-WEBCAM-GENIUS-S-RS-ECAM-8000-RED-NEW_ver_fc0

In [227]:
df

,title,sku,id,category,categoryId,brand,brandId,mainImage,brandImage,initialB,...,categoryIdUser,categoryDescriptionUser,utility,highAverage,widthAverage,lengthAverage,weightAverage,price,warranty,cotizacion
0,ACCESORIOS ADATA CARRY GABINETE P/SSD ED600 BLACK,AED600-U31-CBK,103314,ACCESORIOS,20,ADATA,91,https://static.nb.com.ar/i/nb_ACCESORIOS-ADATA...,https://static.nb.com.ar/img/c2e0fde53fb8cc3bb...,5,...,NaN,NaN,NaN,140.0,243.0,140.0,500,"{'value': 16.95935, 'iva': 10.5, 'finalPrice':...",12 meses,950.5
1,ACCESORIOS ADATA carry GABINETE P/SSD EX500 RED,AEX500U3-CRD,102857,ACCESORIOS,20,ADATA,91,https://static.nb.com.ar/i/nb_ACCESORIOS-ADATA...,https://static.nb.com.ar/img/c2e0fde53fb8cc3bb...,5,...,NaN,NaN,NaN,140.0,243.0,140.0,500,"{'value': 16.3281, 'iva': 10.5, 'finalPrice': ...",12 meses,950.5
2,ACCESORIOS BASE AURICULARES TRUST GXT 265 CINT...,23647,104935,ACCESORIOS,20,TRUST,128,https://static.nb.com.ar/i/nb_ACCESORIOS-BASE-...,https://static.nb.com.ar/img/0d580a29c679330cf...,5,...,NaN,NaN,NaN,140.0,243.0,140.0,500,"{'value': 38.42244, 'iva': 21, 'finalPrice': 4...",12 meses,950.5
3,ACCESORIOS BYH MONITOR PORTATIL PARA NOTEBOOK ...,101-1007P06,118974,ACCESORIOS,20,SENTEY,55,https://static.nb.com.ar/i/nb_ACCESORIOS-BYH-M...,https://static.nb.com.ar/img/e9b0fc242fd226ed6...,5,...,NaN,NaN,NaN,140.0,243.0,140.0,500,"{'value': 284.66718, 'iva': 10.5, 'finalPrice'...",12 meses,950.5
4,ACCESORIOS BYH MONITOR PORTATIL PARA NOTEBOOK ...,101-1010P01,118979,ACCESORIOS,20,SENTEY,55,https://static.nb.com.ar/i/nb_ACCESORIOS-BYH-M...,https://static.nb.com.ar/img/e9b0fc242fd226ed6...,5,...,NaN,NaN,NaN,140.0,243.0,140.0,500,"{'value': 216.91713, 'iva': 10.5, 'finalPrice'...",12 meses,950.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3140,WEBCAM TRUST TRINO HD,18679,104666,WEBCAMS,24,TRUST,128,https://static.nb.com.ar/i/nb_WEBCAM-TRUST-TRI...,https://static.nb.com.ar/img/0d580a29c679330cf...,5,...,NaN,NaN,NaN,200.0,100.0,200.0,300,"{'value': 33.22087, 'iva': 21, 'finalPrice': 4...",12 meses,950.5
3141,WEBCAM TRUST TYRO FULL HD,23637,108604,WEBCAMS,24,TRUST,128,https://static.nb.com.ar/i/nb_WEBCAM-TRUST-TYR...,https://static.nb.com.ar/img/0d580a29c679330cf...,5,...,NaN,NaN,NaN,200.0,100.0,200.0,300,"{'value': 48.32568, 'iva': 21, 'finalPrice': 5...",12 meses,950.5
3142,WEBCAM TRUST VERO FULL HD 1080P GXT1160,22397,104668,WEBCAMS,24,TRUST,128,https://static.nb.com.ar/i/nb_WEBCAM-TRUST-VER...,https://static.nb.com.ar/img/0d580a29c679330cf...,5,...,NaN,NaN,NaN,200.0,100.0,200.0,300,"{'value': 44.52072, 'iva': 21, 'finalPrice': 5...",12 meses,950.5
3143,WEBCAM VIDLOK W77 1080P HD SIN TRIPODE,WEBCAM77,111651,WEBCAMS,24,VIDLOK,213,https://static.nb.com.ar/i/nb_WEBCAM-VIDLOK-W7...,213VL-logo.png,5,...,NaN,NaN,NaN,200.0,100.0,200.0,300,"{'value': 24.38862, 'iva': 21, 'finalPrice': 2...",12 meses,950.5


In [228]:
from sqlalchemy import create_engine, text,event
import urllib

server = 'sobrecodigo-test.database.windows.net'
database = 'dev_2023-06-11T19-20Z'
username = 'adminsobrecodigo'
password = 'Talar2023'

connection_string = (f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+18+for+SQL+Server")

engine = create_engine(connection_string)


In [229]:
categories = pd.read_json(categories_response.text)

C:\Users\Lautaro\AppData\Local\Temp\ipykernel_23792\283115960.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  categories = pd.read_json(categories_response.text)


In [230]:
df[['value','iva','finalPrice','percepcion','finalPriceWithUtility','ncostoextra']]=df.price.apply(pd.Series)
df.drop(columns=['price'],inplace=True)

In [231]:
df.columns = [column.lower() for column in df.columns]

In [232]:
df.to_sql('inventory',schema='nb', con=engine, if_exists='replace', index=False)
categories.to_sql('categories',schema='nb', con=engine, if_exists='replace', index=False)


46